# Программирование на языке Python
## Семинар 25. Функции, замыкание и декораторы
Полезная информация для понимания темы:

1. [Функции - это объекты первого класса](https://habr.com/ru/companies/otus/articles/725374/);
2. [Декораторы](https://habr.com/ru/companies/otus/articles/727590/);
3. [Некоторые встроенные декораторы](https://www.geeksforgeeks.org/top-python-built-in-decorators-that-optimize-python-code-significantly/).

#### План занятия
1. Немного теории;
2. Некоторые полезные декораторы;
3. Практикум: настроим логи с помощью декоратора.

#### Лексическое замыкание
Это способность функции "запоминать" состояние родительской функции.

Рассмотрим на примере:

In [1]:
def enclosing_function():  # функция высшего порядка
    def show_local_state():
        return 'local state'

    return show_local_state

In [3]:
fun_from_fun = enclosing_function()  # функция сама по себе может быть возвращена другой функцией
fun_from_fun()

'local state'

In [4]:
enclosing_function

<function __main__.enclosing_function()>

In [5]:
fun_from_fun

<function __main__.enclosing_function.<locals>.show_local_state()>

In [7]:
callable(4)

False

In [8]:
class SomeCallable:
    def __call__(self):
        return 'you called your callable object'

In [10]:
sc = SomeCallable()
sc()

'you called your callable object'

In [11]:
callable(sc)

True

In [12]:
hasattr(sc, '__call__')

True

**Задание 1**

Поменяйте предложенный ниже код, перенеся условную конструкцию в функцию.

In [14]:
const = 48

def int1_to_str(number: int) -> str:  # baseline function
    return chr(number + const)

def int_to_str(number: int) -> str:  # while
    a = abs(number)

    if number != 0:
        result = ''
        
        while a != 0:  # a > 0
            num = a % 10
            result += int1_to_str(num)  # result = result + int1_to_str(num)
            a //= 10  # a = a // 10
        
        result = result[::-1]  # result = '-' * (some_int < 0) + result[::-1]
        
        if number < 0:
            result = '-' + result

    else:
        result = int1_to_str(number)  # result = '0'
    
    return result

def int_to_str_r(number: int) -> str:  # recursion
    if number < 10:
        return int1_to_str(number)
    else:
        return int_to_str_r(number // 10) + int1_to_str(number % 10)


# select appropriate function depending on number size
number = 892834282939493932

if number // 1e6 > 0:
    result = int_to_str(number)
else:
    result = int_to_str_r(number)

print(result)

892834282939493932


In [15]:
def get_converter(number):
    const = 48

    def int1_to_str(number: int) -> str:  # baseline function
        return chr(number + const)
    
    def int_to_str(number: int) -> str:  # while
        a = abs(number)
    
        if number != 0:
            result = ''
            
            while a != 0:  # a > 0
                num = a % 10
                result += int1_to_str(num)  # result = result + int1_to_str(num)
                a //= 10  # a = a // 10
            
            result = result[::-1]  # result = '-' * (some_int < 0) + result[::-1]
            
            if number < 0:
                result = '-' + result
    
        else:
            result = int1_to_str(number)  # result = '0'
        
        return result
    
    def int_to_str_r(number: int) -> str:  # recursion
        if number < 10:
            return int1_to_str(number)
        else:
            return int_to_str_r(number // 10) + int1_to_str(number % 10)

    if number // 1e6 > 0:
        return int_to_str
    else:
        return int_to_str_r 

In [16]:
converter = get_converter(number)

In [17]:
converter

<function __main__.get_converter.<locals>.int_to_str(number: int) -> str>

In [18]:
converter(number)

'892834282939493932'

Теперь рассмотрим непосредственно замыкание:

In [20]:
def enclosing_function(mode):  # функция высшего порядка
    if mode == 1:
        local_state = 'local state 1'
    elif mode == 2:
        local_state = 'local state 2'
        
    def show_local_state():  # функция "вытаскивает в себя" локальное состояние функции enclosing_function через return
        return local_state

    return show_local_state

In [21]:
local_func = enclosing_function(1)
local_func()

'local state 1'

In [22]:
local_func

<function __main__.enclosing_function.<locals>.show_local_state()>

In [23]:
local_func = enclosing_function(2)
local_func()

'local state 2'

In [24]:
local_func

<function __main__.enclosing_function.<locals>.show_local_state()>

**Задание 2**

Реализуйте ООП-решение задачи предыдущего семинара в парадигме функционального программирования. Обязательно используйте замыкание.

In [35]:
import numpy as np
import pandas as pd

def get_transformer(ttype='std'):
    _valid_structures = [list, tuple, np.ndarray, pd.DataFrame]
    _valid_dtypes = [int, float]

    # fork
    if ttype == 'std':
        from std_functions import get_params, apply_params
    else:
        from norm_functions import get_params, apply_params

    def validate_data(data):
        # check data structure
        structure = type(data)
        if structure not in _valid_structures:
            raise ValueError(f'invalid structure {structure}, use only authorized structures: {_valid_structures}!')
        
        # <data> -> np.ndarray
        if structure in [list, tuple]:
            data = np.array(data)
        elif structure == pd.DataFrame:
            data = data.values
        
        # check dtype
        if data.dtype not in _valid_dtypes:
            raise ValueError(f'invalid dtype {data.dtype}, use only authorized dtypes: {_valid_dtypes}!')

        # shape
        if data.ndim != 2:
            raise ValueError(f'invalid shape {data.ndim}, must be 2!')

        return data

    def fit(data):
        # validate data
        data_valid = validate_data(data)
        
        # compute params
        params = get_params(data_valid)
            
        # save params
        return params

    def transform(data, params) -> pd.DataFrame:
        # validate data
        data_valid = validate_data(data)

        # apply params to data
        data_transformed = apply_params(data_valid, params)

        return data_transformed

    return fit, transform

In [36]:
X = np.random.uniform(-10, 10, (100, 2))
y = np.random.normal(0, 1, 100)

X_new = np.random.uniform(-10, 10, (50, 2))

# main.py
fit, transform = get_transformer()

params = fit(X)
X_transformed = transform(X, params)

In [37]:
X_transformed

array([[ 0.17799244, -1.21908303],
       [-0.9613307 ,  1.73313658],
       [ 1.09113191,  0.81234865],
       [ 0.89466438,  0.52373211],
       [-0.52581817,  0.98691854],
       [-1.52767193,  0.93017996],
       [ 1.32084349, -0.72570074],
       [-0.54925808,  0.55226873],
       [ 0.72186514,  1.20508598],
       [-0.00489674,  1.87178796],
       [ 1.46746842,  0.65245267],
       [ 1.12125366, -0.25405326],
       [ 0.58458188, -1.09847009],
       [-1.32104969, -0.41330441],
       [ 1.44213494,  0.62396097],
       [-0.09006467, -1.48701157],
       [-0.66563935, -0.34649125],
       [ 1.00961818, -0.14224949],
       [-0.85206454,  0.15113212],
       [-1.10470645,  1.29885297],
       [ 0.92164715,  1.35331364],
       [ 0.37163103,  1.40726751],
       [-1.33195924,  0.32540815],
       [-0.57733473,  1.79568603],
       [ 0.40539095,  1.7730987 ],
       [ 1.65169132, -0.30928663],
       [ 0.89268624,  0.51670355],
       [ 0.97959194, -0.65045565],
       [-0.32579379,

#### Декораторы
Декоратор - это, в сущности, функция, которая принимает на вход другую функцию и возвращает ее модификацию:

In [38]:
from datetime import datetime

def what_time_is_it():
    now = datetime.now()

    return now

In [39]:
what_time_is_it

<function __main__.what_time_is_it()>

In [40]:
what_time_is_it()

datetime.datetime(2023, 11, 27, 20, 32, 46, 279650)

In [41]:
def prettifier(func):
    def wrapper():
        now = func()
        pretty_now = now.strftime('%d.%m.%Y %H:%M:%S')
        
        return pretty_now
        
    return wrapper

**БЕЗ СИНТАКСИЧЕСКОГО САХАРА:**

In [43]:
def what_time_is_it():
    now = datetime.now()

    return now

what_time_is_it = prettifier(what_time_is_it)

In [44]:
what_time_is_it

<function __main__.prettifier.<locals>.wrapper()>

**С СИНТАКСИЧЕСКИМ САХАРОМ:**

In [47]:
def what_time_is_it():
    now = datetime.now()

    return now

In [48]:
@prettifier
def what_time_is_it():
    now = datetime.now()

    return now

In [49]:
what_time_is_it

<function __main__.prettifier.<locals>.wrapper()>

In [50]:
what_time_is_it()

'27.11.2023 20:38:08'

А как с аргументами?

In [51]:
from pathlib import Path
import openpyxl

def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
        if row[0].value != 'Блок'
    }

    wb.close()

    return info_dict

In [52]:
info_path = Path('../Занятие 5/sources/Справочник.xlsx')

parse_info(info_path)

{('Блок 1', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент D'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент E'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент F'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 

In [63]:
import pandas as pd

def pandify(func):
    def wrapper(*args, **kwargs):
        dictlike = func(*args, **kwargs)
        
        list_to_pandas = [{'block': key[0], 'department': key[1], **value} for key, value in dictlike.items()]

        df = pd.DataFrame(list_to_pandas)

        return df

    return wrapper

In [55]:
def std(*args, ddof=0):
    mean = sum(args) / len(args)
    std = (sum([(xi - mean) ** 2 for xi in args]) / (len(args) - ddof)) ** (1 / 2)
    
    return std

In [60]:
args = [6, 7, 8]
kwargs = {'ddof': 1}

In [62]:
std(*args, **kwargs)

1.0

In [58]:
std(1, 2, 3, 4, ddof=1)

1.2909944487358056

In [64]:
@pandify
def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
        if row[0].value != 'Блок'
    }

    wb.close()

    return info_dict

In [65]:
info_path = Path('../Занятие 5/sources/Справочник.xlsx')

parse_info(info_path)

,block,department,fio,email
0,Блок 1,Департамент A,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
1,Блок 1,Департамент B,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
2,Блок 1,Департамент C,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
3,Блок 2,Департамент A,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
4,Блок 2,Департамент B,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
5,Блок 2,Департамент C,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
6,Блок 2,Департамент D,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
7,Блок 2,Департамент E,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
8,Блок 2,Департамент F,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
9,Блок 2,Департамент G,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com


**Как быть с docstring?**

In [66]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [67]:
func.__doc__

'\n    This function powers input n by 2\n    '

In [69]:
def divider(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)
    return wrapper

In [71]:
@divider
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [72]:
?func

Signature: func(*args, **kwargs)
Docstring: <no docstring>
File:      /var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_6350/3284642871.py
Type:      function

**Решение**

In [73]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [75]:
import functools

def divider(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)
    return wrapper

In [76]:
?func

Signature: func(n)
Docstring: This function powers input n by 2
File:      /var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_6350/855151311.py
Type:      function

#### Некоторые полезные декораторы

1. @classmethod (метод для класса, подразумевает аргумент cls);
2. @staticmethod (метод для класса, не привязан классу никак кроме namespace);
3. @abstractmethod (только обозначить наличие метода);
4. @typing.final (обозначить, что от класса нельзя наследоваться);
5. @lru_cache.

In [78]:
from abc import ABC, abstractmethod

In [85]:
class Transformer(ABC):
    @abstractmethod
    def _get_params(self, data):  # must be specified in child class
        pass
        
    @abstractmethod
    def _apply_params(self, data):  # must be specified in child class
        pass

In [86]:
tr = Transformer()

TypeError: Can't instantiate abstract class Transformer with abstract methods _apply_params, _get_params

#### Практика
Напишите декоратор для сохранения логов в файл stdout.txt. Модифицируйте проект по рассылке отчетов.

In [ ]:
# наш код здесь

In [88]:
def printer(number):
    print(f'value {number} printed successfully')

def printer2(number):
    print(f'value {number} multiplied by 2 successfully: {number * 2}')

In [89]:
number = 10

printer(number)
printer2(number)

value 10 printed successfully
value 10 multiplied by 2 successfully: 20


In [91]:
import sys

sys.stdout

In [92]:
def cache_log(func):
    def wrapper(*args, **kwargs):
        original_stdout = sys.stdout

        with open('stdout.txt', 'a') as new_stdout:
            sys.stdout = new_stdout
            func(*args, **kwargs)

        sys.stdout = original_stdout
    
    return wrapper

In [93]:
@cache_log
def printer(number):
    print(f'value {number} printed successfully')

@cache_log
def printer2(number):
    print(f'value {number} multiplied by 2 successfully: {number * 2}')

In [94]:
number = 10

printer(number)
printer2(number)

In [95]:
print(1)

1
